# Imports and loading datasets

In [ ]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

I'm going to use `tf.estimator` API.

## Input functions

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=256,
    num_epochs=None,
    shuffle=True    
)

In [ ]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_val},
    y=y_val,
    num_epochs=1,
    shuffle=False    
)

In [ ]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False    
)

In [ ]:
train_test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=256,
    num_epochs=1,
    shuffle=True    
)

## CNN of my own design

In [ ]:
def my_cnn(features, labels, mode):
    
    X = tf.cast(features["x"], tf.float32)    
    reg = tf.contrib.layers.l2_regularizer(scale=0.01)
    # convolutional layers
    h1 = tf.layers.conv2d(X, 32, 3, activation=tf.nn.relu, kernel_regularizer=reg)
    h1_pool = tf.layers.max_pooling2d(h1, (2, 2), (2, 2))
    h1_batchnorm = tf.layers.batch_normalization(h1_pool)
    
    h2 = tf.layers.conv2d(h1_batchnorm, 64, 2, activation=tf.nn.relu, kernel_regularizer=reg)
    h2_pool = tf.layers.max_pooling2d(h2, (2, 2), (2, 2))
    h2_batchnorm = tf.layers.batch_normalization(h2_pool)
    
    h3 = tf.layers.conv2d(h2_batchnorm, 128, 2, activation=tf.nn.relu, kernel_regularizer=reg)
    h3_batchnorm = tf.layers.batch_normalization(h3)
    # dense layers
    d0 = tf.layers.flatten(h3_batchnorm)
    
    d1 = tf.layers.dense(d0, 1024, activation=tf.nn.relu, kernel_regularizer=reg)
    d1_batchnorm = tf.layers.batch_normalization(d1)    
    d2 = tf.layers.dense(d1_batchnorm, 10, kernel_regularizer=reg)
    
    y_out = d2
        
    predictions = {
        "classes": tf.argmax(input=y_out, axis=1),
        "probabilities": tf.nn.softmax(y_out, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    l2_loss = tf.losses.get_regularization_loss()
    onehot_labels = tf.one_hot(indices=labels, depth=10)
    softmax_loss = tf.losses.softmax_cross_entropy(logits=y_out, onehot_labels=onehot_labels)
    loss = l2_loss + softmax_loss
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric)

In [ ]:
my_cnn_classifier = tf.estimator.Estimator(model_fn=my_cnn, model_dir='/tmp/my_cnn')

In [ ]:
my_cnn_classifier.train(input_fn=train_input_fn, steps=3000)

In [ ]:
test_results = my_cnn_classifier.evaluate(input_fn=test_input_fn)
val_results = my_cnn_classifier.evaluate(input_fn=val_input_fn)
train_results = my_cnn_classifier.evaluate(input_fn=train_test_input_fn)
print('val', val_results)
print('test', test_results)
print('train', train_results)


Measured accuracy:
* Validation set: 73.0%
* Test set: 71.3%
* Training set: 85.9%